# Automatic generation of Notebook using PyCropML
    This notebook implements a crop model.

### Model Phyllochron

In [ ]:
MODULE list_sub
    IMPLICIT NONE
    TYPE container
        CLASS(*), ALLOCATABLE :: item
        CLASS(*), ALLOCATABLE :: items(:)
    END TYPE

    interface Add
        module procedure AddToListFloat
        module procedure AddToListInt
        module procedure AddToListChar
        module procedure AddToListIntArray
    end interface
CONTAINS

    FUNCTION indice(vectorElem, elem)
        CHARACTER(LEN=*), DIMENSION(:):: vectorElem
        INTEGER::iterator, indice
        CHARACTER(LEN=*):: elem
        DO iterator= 1, SIZE(vectorElem)
            IF(vectorElem(iterator)==elem) THEN
                indice = iterator
            END IF
        END DO
        RETURN
    END FUNCTION indice


    SUBROUTINE AddToListFloat(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize
        REAL, INTENT(IN) :: element
        REAL, DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        REAL, DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            ALLOCATE(list(1))
            list(1) = element
        END IF
    END SUBROUTINE AddToListFloat

    SUBROUTINE AddToListInt(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize
        INTEGER, INTENT(IN) :: element
        INTEGER, DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        INTEGER, DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            ALLOCATE(list(1))
            list(1) = element
        END IF
    END SUBROUTINE AddToListInt

    SUBROUTINE AddToListChar(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize, l
        CHARACTER(LEN=*), INTENT(IN) :: element
        CHARACTER(LEN=*), DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        CHARACTER(LEN=65), DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            l=1
            ALLOCATE(list(l))
            list(l) = element
        END IF
    END SUBROUTINE AddToListChar


    SUBROUTINE AddToListIntArray(a, e)
        INTEGER, ALLOCATABLE, INTENT(INOUT) :: a(:)
        INTEGER, ALLOCATABLE, INTENT(IN) :: e(:)
        INTEGER, ALLOCATABLE:: tmp(:)

        IF (.NOT.ALLOCATED(a)) THEN
            ALLOCATE(a(1))
            ALLOCATE(a(SIZE(e)), source = e)
        ELSE
            CALL MOVE_ALLOC(a,tmp)
            ALLOCATE(a(SIZE(tmp)+SIZE(e)), source = e)
            a(1:SIZE(tmp)) = tmp
            a(SIZE(tmp)+1:SIZE(a)) = e
        END IF
    END SUBROUTINE AddToListIntArray

END MODULE list_sub
MODULE Phyllochronmod
    IMPLICIT NONE
CONTAINS

    SUBROUTINE model_phyllochron(fixPhyll, &
        leafNumber, &
        lincr, &
        ldecr, &
        pdecr, &
        pincr, &
        ptq, &
        gAImean, &
        kl, &
        pTQhf, &
        B, &
        p, &
        choosePhyllUse, &
        areaSL, &
        areaSS, &
        lARmin, &
        lARmax, &
        sowingDensity, &
        lNeff, &
        phyllochron)
        IMPLICIT NONE
        REAL, INTENT(IN) :: fixPhyll
        REAL, INTENT(IN) :: leafNumber
        REAL, INTENT(IN) :: lincr
        REAL, INTENT(IN) :: ldecr
        REAL, INTENT(IN) :: pdecr
        REAL, INTENT(IN) :: pincr
        REAL, INTENT(IN) :: ptq
        REAL, INTENT(IN) :: gAImean
        REAL, INTENT(IN) :: kl
        REAL, INTENT(IN) :: pTQhf
        REAL, INTENT(IN) :: B
        REAL, INTENT(IN) :: p
        CHARACTER(65), INTENT(IN) :: choosePhyllUse
        REAL, INTENT(IN) :: areaSL
        REAL, INTENT(IN) :: areaSS
        REAL, INTENT(IN) :: lARmin
        REAL, INTENT(IN) :: lARmax
        REAL, INTENT(IN) :: sowingDensity
        REAL, INTENT(IN) :: lNeff
        REAL, INTENT(OUT) :: phyllochron
        REAL:: gaiLim
        REAL:: LAR
        !- Description:
    !            * Title: Phyllochron Model
    !            * Author: Pierre Martre
    !            * Reference: Modeling development phase in the 
    !                Wheat Simulation Model SiriusQuality.
    !                See documentation at http://www1.clermont.inra.fr/siriusquality/?page_id=427
    !            * Institution: INRA Montpellier
    !            * Abstract: Calculate different types of phyllochron 
        !- inputs:
    !            * name: fixPhyll
    !                          ** description : Sowing date corrected Phyllochron
    !                          ** inputtype : variable
    !                          ** variablecategory : auxiliary
    !                          ** datatype : DOUBLE
    !                          ** default : 5.0
    !                          ** min : 0.0
    !                          ** max : 10000.0
    !                          ** unit : °C d leaf-1
    !                          ** uri : some url
    !            * name: leafNumber
    !                          ** description : Actual number of phytomers
    !                          ** inputtype : variable
    !                          ** variablecategory : state
    !                          ** datatype : DOUBLE
    !                          ** default : 0.0
    !                          ** min : 0.0
    !                          ** max : 25.0
    !                          ** unit : leaf
    !                          ** uri : some url
    !            * name: lincr
    !                          ** description : Leaf number above which the phyllochron is increased by Pincr
    !                          ** inputtype : parameter
    !                          ** parametercategory : species
    !                          ** datatype : DOUBLE
    !                          ** default : 8.0
    !                          ** min : 0.0
    !                          ** max : 30.0
    !                          ** unit : leaf
    !                          ** uri : some url
    !            * name: ldecr
    !                          ** description : Leaf number up to which the phyllochron is decreased by Pdecr
    !                          ** inputtype : parameter
    !                          ** parametercategory : species
    !                          ** datatype : DOUBLE
    !                          ** default : 0.0
    !                          ** min : 0.0
    !                          ** max : 100.0
    !                          ** unit : leaf
    !                          ** uri : some url
    !            * name: pdecr
    !                          ** description : Factor decreasing the phyllochron for leaf number less than Ldecr
    !                          ** inputtype : parameter
    !                          ** parametercategory : species
    !                          ** datatype : DOUBLE
    !                          ** default : 0.4
    !                          ** min : 0.0
    !                          ** max : 10.0
    !                          ** unit : -
    !                          ** uri : some url
    !            * name: pincr
    !                          ** description : Factor increasing the phyllochron for leaf number higher than Lincr
    !                          ** inputtype : parameter
    !                          ** parametercategory : species
    !                          ** datatype : DOUBLE
    !                          ** default : 1.5
    !                          ** min : 0.0
    !                          ** max : 10.0
    !                          ** unit : -
    !                          ** uri : some url
    !            * name: ptq
    !                          ** description : Photothermal quotient 
    !                          ** inputtype : variable
    !                          ** variablecategory : state
    !                          ** datatype : DOUBLE
    !                          ** default : 0.0
    !                          ** min : 0.0
    !                          ** max : 10000.0
    !                          ** unit : MJ °C-1 d-1 m-2)
    !                          ** uri : some url
    !            * name: gAImean
    !                          ** description : Green Area Index
    !                          ** inputtype : variable
    !                          ** variablecategory : state
    !                          ** datatype : DOUBLE
    !                          ** default : 0.0
    !                          ** min : 0.0
    !                          ** max : 10000.0
    !                          ** unit : m2 m-2
    !                          ** uri : some url
    !            * name: kl
    !                          ** description : Exctinction Coefficient
    !                          ** inputtype : parameter
    !                          ** parametercategory : species
    !                          ** datatype : DOUBLE
    !                          ** default : 0.45
    !                          ** min : 0.0
    !                          ** max : 50.0
    !                          ** unit : -
    !                          ** uri : some url
    !            * name: pTQhf
    !                          ** description : Slope to intercept ratio for Phyllochron  parametrization with PhotoThermal Quotient
    !                          ** inputtype : parameter
    !                          ** parametercategory : genotypic
    !                          ** datatype : DOUBLE
    !                          ** default : 0.0
    !                          ** min : 0.0
    !                          ** max : 1000.0
    !                          ** unit : °C d leaf-1
    !                          ** uri : some url
    !            * name: B
    !                          ** description : Phyllochron at PTQ equal 1
    !                          ** inputtype : parameter
    !                          ** parametercategory : species
    !                          ** datatype : DOUBLE
    !                          ** default : 20.0
    !                          ** min : 0.0
    !                          ** max : 1000.0
    !                          ** unit : °C d leaf-1
    !                          ** uri : some url
    !            * name: p
    !                          ** description : Phyllochron (Varietal parameter)
    !                          ** inputtype : parameter
    !                          ** parametercategory : species
    !                          ** datatype : DOUBLE
    !                          ** default : 120.0
    !                          ** min : 0.0
    !                          ** max : 1000.0
    !                          ** unit : °C d leaf-1
    !                          ** uri : some url
    !            * name: choosePhyllUse
    !                          ** description : Switch to choose the type of phyllochron calculation to be used
    !                          ** inputtype : parameter
    !                          ** parametercategory : species
    !                          ** datatype : STRING
    !                          ** default : Default
    !                          ** min : 
    !                          ** max : 
    !                          ** unit : -
    !                          ** uri : some url
    !            * name: areaSL
    !                          ** description :  Area Leaf
    !                          ** inputtype : parameter
    !                          ** parametercategory : genotypic
    !                          ** datatype : DOUBLE
    !                          ** default : 0.0
    !                          ** min : 0.0
    !                          ** max : 1000.0
    !                          ** unit : cm2
    !                          ** uri : some url
    !            * name: areaSS
    !                          ** description : Area Sheath
    !                          ** inputtype : parameter
    !                          ** parametercategory : genotypic
    !                          ** datatype : DOUBLE
    !                          ** default : 0.0
    !                          ** min : 0.0
    !                          ** max : 1000.0
    !                          ** unit : cm2
    !                          ** uri : some url
    !            * name: lARmin
    !                          ** description : LAR minimum
    !                          ** inputtype : parameter
    !                          ** parametercategory : genotypic
    !                          ** datatype : DOUBLE
    !                          ** default : 0.0
    !                          ** min : 0.0
    !                          ** max : 1000.0
    !                          ** unit : leaf-1 °C
    !                          ** uri : some url
    !            * name: lARmax
    !                          ** description : LAR maximum
    !                          ** inputtype : parameter
    !                          ** parametercategory : genotypic
    !                          ** datatype : DOUBLE
    !                          ** default : 0.0
    !                          ** min : 0.0
    !                          ** max : 1000.0
    !                          ** unit : leaf-1 °C
    !                          ** uri : some url
    !            * name: sowingDensity
    !                          ** description : Sowing Density
    !                          ** inputtype : parameter
    !                          ** parametercategory : genotypic
    !                          ** datatype : DOUBLE
    !                          ** default : 0.0
    !                          ** min : 0.0
    !                          ** max : 1000.0
    !                          ** unit : plant m-2
    !                          ** uri : some url
    !            * name: lNeff
    !                          ** description : Leaf Number efficace
    !                          ** inputtype : parameter
    !                          ** parametercategory : genotypic
    !                          ** datatype : DOUBLE
    !                          ** default : 0.0
    !                          ** min : 0.0
    !                          ** max : 1000.0
    !                          ** unit : leaf
    !                          ** uri : some url
        !- outputs:
    !            * name: phyllochron
    !                          ** description :  the rate of leaf appearance 
    !                          ** variablecategory : state
    !                          ** datatype : DOUBLE
    !                          ** min : 0
    !                          ** max : 1000
    !                          ** unit :  °C d leaf-1
    !                          ** uri : some url
        phyllochron = 0.0
        LAR = 0.0
        gaiLim = lNeff * ((areaSL + areaSS) / 10000.0) * sowingDensity
        IF(choosePhyllUse .EQ. 'Default') THEN
            IF(leafNumber .LT. ldecr) THEN
                phyllochron = fixPhyll * pdecr
            ELSE IF ( leafNumber .GE. ldecr .AND. leafNumber .LT. lincr) THEN
                phyllochron = fixPhyll
            ELSE
                phyllochron = fixPhyll * pincr
            END IF
        END IF
        IF(choosePhyllUse .EQ. 'PTQ') THEN
            IF(gAImean .GT. gaiLim) THEN
                LAR = (lARmin + ((lARmax - lARmin) * ptq / (pTQhf + ptq))) / (B *  &
                        gAImean)
            ELSE
                LAR = (lARmin + ((lARmax - lARmin) * ptq / (pTQhf + ptq))) / (B *  &
                        gaiLim)
            END IF
            phyllochron = 1.0 / LAR
        END IF
        IF(choosePhyllUse .EQ. 'Test') THEN
            IF(leafNumber .LT. ldecr) THEN
                phyllochron = p * pdecr
            ELSE IF ( leafNumber .GE. ldecr .AND. leafNumber .LT. lincr) THEN
                phyllochron = p
            ELSE
                phyllochron = p * pincr
            END IF
        END IF
    END SUBROUTINE model_phyllochron

END MODULE
PROGRAM test
    USE Phyllochronmod
    REAL:: fixPhyll
    REAL:: leafNumber
    REAL:: lincr
    REAL:: ldecr
    REAL:: pdecr
    REAL:: pincr
    REAL:: ptq
    REAL:: gAImean
    REAL:: kl
    REAL:: pTQhf
    REAL:: B
    REAL:: p
    CHARACTER(65):: choosePhyllUse
    REAL:: areaSL
    REAL:: areaSS
    REAL:: lARmin
    REAL:: lARmax
    REAL:: sowingDensity
    REAL:: lNeff
    REAL:: phyllochron
    print *, "--------test_test_wheat1_Phyllochron-------"
    lincr = 8.0
    ldecr = 3.0
    pdecr = 0.4
    pincr = 1.25
    ptq = 0.97
    kl = 0.45
    p = 120.0
    choosePhyllUse = "Default"
    fixPhyll = 91.2
    leafNumber = 0.0
    gAImean = 0.0
    pTQhf = 0.0
    B = 20.0
    areaSL = 0.0
    areaSS = 0.0
    lARmin = 0.0
    lARmax = 0.0
    sowingDensity = 0.0
    lNeff = 0.0
    call model_phyllochron(fixPhyll, leafNumber, lincr, ldecr, pdecr,  &
            pincr, ptq, gAImean, kl, pTQhf, B, p, choosePhyllUse, areaSL, areaSS,  &
            lARmin, lARmax, sowingDensity, lNeff, phyllochron)
    !phyllochron: 36.48
    print *, "phyllochron estimated :"
    print *, phyllochron

END PROGRAM
